In [ ]:
from IPython.display import display, Markdown, HTML
import logging
import random

In [3]:
# Suppress INFO messages from httpx as there can be distracting messages that aren't relevant to the user
logging.getLogger("httpx").setLevel(logging.ERROR)

class Chatbot:
    def __init__(self, model_name, system_role_content, max_inputs, tone, input_length_limit=4000):
        # set instance variables
        self.max_inputs = max_inputs
        self.input_length_limit = input_length_limit
        # Initialize history list to store the conversation history
        self.history = []
        # Initialize input count to keep track of the number of inputs
        self.input_count = 0

        # Set tone parameters by calling set_tone method defined below
        self.set_tone(tone)
        
        from class_version import OpenAIClient
        # Initialize the AI instance with the tone parameters
        self.ai = OpenAIClient(
            model_name=model_name,
            system_role_content=system_role_content,
            
            # use the values set by set_tone method 
            temperature=self.temperature,
            top_p=self.top_p
        )

    def set_tone(self, tone):
        
        # set temperature and top_p based on the tone by choosing random value within the given range
        if tone == "creative":
            self.temperature = random.uniform(0.8, 1.35)
            self.top_p = random.uniform(0.8, 1.0)
        elif tone == "balanced":
            # there is a buffer zone of .7 to .8 to create a defined difference between balanced and creative
            self.temperature = random.uniform(0.4, 0.7)
            self.top_p = random.uniform(0.4, 0.7)
        elif tone == "precise":
            # there is a buffer zone of .3 to .4 to create a defined difference between balanced and precise
            self.temperature = random.uniform(0.0, 0.3)
            self.top_p = random.uniform(0.0, 0.3)
        else:
            raise ValueError("Invalid tone. Choose from 'creative', 'balanced', or 'precise'.")
        
        
        
    def chat(self):
        # loop to continue the conversation until the max_inputs is reached or the user types "exit" or "quit"
        while self.input_count < self.max_inputs:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            if len(user_input.split()) > self.input_length_limit:
                display(Markdown(f"__Error: Input exceeds the length limit of {self.input_length_limit} words.__"))
                # skip the current iteration and continue with the next iteration
                continue

            # Display user prompt
            display(HTML(f'''
                        <article style="background-color: rgb(70, 70, 70); margin: 10px; padding: 10px; height: auto; 
                        line-height: 1.5; width: 75%"><b>You:</b> {user_input}</article>
                        '''))

            # Add user input to history
            self.history.append(f"User: {user_input}")

            # Create a prompt with the conversation history by using the join method to concatenate the list of prompts and responses
            prompt = "\n".join(self.history) + f"\nAI:"

            # Get the AI response
            response = self.ai.get_response(prompt)

            # Add AI response to history
            self.history.append(f"AI: {response}")

            # Increment the input count
            self.input_count += 1
            
            # Display AI response
            display(Markdown(f"__AI:__ {response}"))
            
            if self.input_count >= self.max_inputs:
              display(Markdown(f"### __This conversation has reached the limit of {self.max_inputs} inputs.__"))
              
        print("Saving file....") 
        print("This may take a few moments.")
        self.save_conversation()
        
        
        
    def save_conversation(self):
        from class_version import OpenAIClient
        self.ai = OpenAIClient(
            model_name='gpt-4o-mini',
            system_role_content="You are a helpful assistant.",
            temperature=0.1,
            top_p=0.1
        )
        name = self.ai.get_response(f"""The following is a record of a conversation between a user and an AI assistant: {self.history}
                            Please find a word or a phrase with which to name this conversation to be the name of the file.
                            The name should be no longer than three words and should not contain any special characters or spaces.""")
        
        # Define the directory path
        import os
        directory = "C:/Users/Rebecca/OneDrive/Documents/Python AI/AI Conversations"
        
        # Ensure the directory exists
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # Use the directory path when opening the file
        file_path = os.path.join(directory, name)
        
        try:
            with open(file_path + ".md", "w") as markdown:
                for message in self.history:
                    markdown.write(message + "\n")
                
                    
            with open(file_path + ".md", "r") as markdown:
                md_content = markdown.read()
            
            
            with open('html_formatting.txt', 'r') as html_instructions:  
                instructions = html_instructions.read()
            
                
                html_dev = OpenAIClient(model_name='gpt-4o-mini', 
                                        system_role_content="You are a skilled web developer.", 
                                        temperature=0.1, 
                                        top_p=0.1)
                
                import datetime
                # Current date and time
                date = datetime.datetime.now()

                # Format date and time
                formatted_date = date.strftime("%b %#d, %Y %#I:%M %p")
        
                conversation = html_dev.get_response(f"""Please convert {md_content} to HTML format preserving the markdown formatting it has.
                                        You can use the following code as a template: {instructions}. Pleaes put this as the date in the top right corner: {formatted_date}
                                        The file name should be displayed as an h2 header at the top of the page. This is the name: {name}
                                        """) 
                
            with open(file_path + ".html", "w") as file:
                # Trim the conversation to only include the actual HTML and not the response of the AI
                start_index = conversation.find("<!DOCTYPE html>")
                end_index = conversation.rfind("</html>") + len("</html>")
                conversation = conversation[start_index:end_index]
                file.write(conversation)
                
            # Verify the HTML file was created and has valid content
            if os.path.exists(file_path + ".html") and os.path.getsize(file_path + ".html") > 30:  # Threshold for meaningful content
                os.remove(file_path + ".md")  # Delete the .md file
                extension = ".html"
                
            else:
            # keep the markdown file if the HTML file is empty or has insufficient content
                extension = ".md"
                raise ValueError("HTML file is empty or has insufficient content.")
              
                
                  
        except Exception as e:
            print(f"Error: {e}")
        
        display(Markdown(f"### __The conversation has been saved as {name}.{extension}__"))    



In [4]:
chatbot = Chatbot(
    model_name="gpt-4o",
    system_role_content="""You are a tech expert. """,
    max_inputs=50,
    tone="balanced"
)

chatbot.chat()       

__AI:__ Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?

__AI:__ When you open a Jupyter Notebook, you're prompted to select a kernel, which is essentially the environment in which your code will run. Here's a breakdown of the options you might encounter and when to choose each one:

1. **Python Environments**:
   - **Base Environment**: This is the default Python environment that comes with your Anaconda installation (if you're using Anaconda) or your system's default Python environment. Use this if you haven't set up any specific environments or if your notebook doesn't require any special packages.
   - **Custom Python Environments**: These are environments you've created using tools like `conda` or `virtualenv`. Choose one of these if your notebook requires specific packages or versions that are installed in that environment. For example, if you have a project that relies on a specific version of a library, you should select the environment where that version is installed.

2. **Other Languages (e.g., R, Julia)**:
   - If you have kernels for other languages installed, such as R or Julia, you can choose these if your notebook is written in that language. Jupyter supports multiple languages, so if you're working with a notebook that contains R code, for example, you'll need to select the R kernel.

3. **Specialized Kernels**:
   - **IPython Kernel**: This is the default kernel for Python code in Jupyter. It's a good choice for general Python development.
   - **Other Specialized Kernels**: Sometimes, you might have kernels that are optimized for specific tasks, such as data science or machine learning. These might come pre-installed with certain libraries or tools that are useful for those tasks.

### When to Choose a Specific Kernel:

- **Project-Specific Requirements**: If your project requires specific libraries or versions, choose the environment where those are installed.
- **Language Requirements**: If your notebook is written in a language other than Python, select the appropriate kernel for that language.
- **Consistency Across Projects**: If you're working on multiple projects, it can be helpful to create and use separate environments for each to avoid conflicts between package versions.
- **Performance and Features**: Some environments might be optimized for performance or include additional features that are beneficial for your work.

In general, it's a good practice to create a new environment for each project to manage dependencies more effectively. This way, you can ensure that your code runs consistently and without conflicts.

__AI:__ Got it! Let's focus on the options you see in Jupyter Notebook when selecting a kernel, especially if you're not using Anaconda or Conda environments.

### Understanding Jupyter Notebook Kernel Options:

1. **Python Environments**:
   - **System Python**: This is the default Python installation on your system. If you haven't set up any virtual environments, this is likely what you're using. It's a good choice if you don't have specific package requirements or if you're working on a simple project.
   - **Virtual Environments**: If you've created virtual environments using `venv` or `virtualenv`, these can be used as kernels in Jupyter. To use a virtual environment as a kernel, you need to install the `ipykernel` package within that environment and register it with Jupyter. This is useful for projects with specific dependencies.

2. **Jupyter Kernels**:
   - **IPython Kernel**: This is the standard kernel for running Python code in Jupyter. It's suitable for most Python projects unless you have specific needs that require a different setup.
   - **Other Language Kernels**: If you have installed kernels for other languages (like R or Julia), they will appear as options. Choose these if your notebook is written in that language.

### How to Choose the Right Kernel:

- **Using Virtual Environments**: If you have a virtual environment set up for your project, you should use it to ensure all dependencies are correctly managed. To add a virtual environment as a Jupyter kernel, activate the environment and run:
  ```bash
  pip install ipykernel
  python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"
  ```
  Replace `myenv` with the name of your environment. This will make it appear as an option in Jupyter.

- **Default/System Python**: If you don't have specific dependencies or are working on a simple script, using the system Python or the default IPython kernel is fine.

- **Project-Specific Needs**: If your project has specific requirements (e.g., certain libraries or versions), ensure those are installed in the environment you choose. This might mean setting up a virtual environment specifically for that project.

In summary, if you're not using Conda, managing virtual environments with `venv` or `virtualenv` and setting them up as Jupyter kernels is a good practice for keeping your projects organized and dependencies isolated.

__AI:__ Using your global Python installation for Jupyter Notebooks is not inherently a bad idea, especially if your projects are simple and don't have conflicting dependencies. However, there are some potential downsides and considerations to keep in mind:

### Potential Issues with Using Global Python:

1. **Dependency Conflicts**: If you install multiple packages globally, you might run into version conflicts. For example, one project might require `pandas==1.2.0` while another requires `pandas==1.3.0`. Managing these dependencies globally can become cumbersome and error-prone.

2. **System Stability**: Installing and updating packages globally can sometimes affect system tools or other applications that rely on Python. This is especially true if you're using Python that comes pre-installed with your operating system.

3. **Reproducibility**: If you share your code with others or move it to a different machine, it might not work if the global environment is different. Virtual environments help encapsulate dependencies, making it easier to reproduce the same setup elsewhere.

4. **Clutter**: Over time, your global Python environment can become cluttered with packages that are no longer needed, which can slow down package management operations and increase the risk of conflicts.

### Benefits of Using Virtual Environments:

- **Isolation**: Each project can have its own set of dependencies, which prevents conflicts and makes it easier to manage different projects with different requirements.
- **Reproducibility**: You can easily share your `requirements.txt` or `Pipfile` with others, allowing them to set up the same environment.
- **Cleaner Global Environment**: Your global Python installation remains clean and stable, reducing the risk of system issues.

### Recommendations:

- **For Simple Projects**: If your projects are simple and don't have conflicting dependencies, using the global Python environment is fine. Just be cautious about installing packages that might affect other projects.
  
- **For Complex Projects**: For projects with specific dependencies or if you're working on multiple projects simultaneously, it's a good idea to use virtual environments. They provide a clean and isolated space for each project.

In summary, while using the global Python environment is convenient, virtual environments offer more flexibility and control, especially as your projects grow in complexity. If you haven't encountered issues yet, that's great, but it's worth considering virtual environments as a best practice for future projects.

Saving file....
This may take a few moments.


### __The conversation has been saved as JupyterKernelSelection..html__